In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def filter_and_process_excel(file_path):
    df = pd.read_excel(file_path)
    print(f"Original data shape: {df.shape}")
    print("Columns:", df.columns.tolist())
    order_column = 'Order No.'
    df[order_column] = df[order_column].astype(str)
    filtered_df = df[
        df[order_column].str.startswith('00000007') | 
        df[order_column].str.startswith('00000009')
    ].copy()
    print(f"Filtered data shape: {filtered_df.shape}")
    print(f"Rows removed: {len(df) - len(filtered_df)}")
    filtered_df = filtered_df.dropna(subset=['Posting Date', 'Order No.'])
    filtered_df = filtered_df.sort_values('Posting Date', ascending=True)
    filtered_df = filtered_df.reset_index(drop=True)
    filtered_df = filtered_df.replace({np.nan: None})
    display(filtered_df.head())
    display(filtered_df.tail())
    return filtered_df

file_path = "D:\\WorkingFolder\\OneDrive - vikramsolar.com\\Desktop\\VSL Projects\\QC\\QC_Data\\UD Report_Sep-25.XLSX"
processed_df = filter_and_process_excel(file_path)

Original data shape: (412884, 8)
Columns: ['Posting Date', 'Creation Time', 'Order No.', 'Material Desc.', 'Serial Number', 'Created By', 'Grade.', 'Reason']
Filtered data shape: (296716, 8)
Rows removed: 116168


,Posting Date,Creation Time,Order No.,Material Desc.,Serial Number,Created By,Grade.,Reason
0,2025-09-01,08:56:26,000000071361,PV MODULE D HYPERSOL VSMDH.72.600.05-30M,26062065,BISWAJIT.UD4,D,None
1,2025-09-01,15:31:30,000000090342,PV MODULE D HYPERSOL VSMDH.72.595.05-30M,26060983,KRISH.SGR2,D,None
2,2025-09-01,15:31:32,000000090342,PV MODULE D HYPERSOL VSMDH.72.595.05-30M,26060987,KRISH.SGR2,D,None
3,2025-09-01,15:31:34,000000090342,PV MODULE D HYPERSOL VSMDH.72.595.05-30M,26060992,KRISH.SGR2,D,None
4,2025-09-01,15:31:36,000000090342,PV MODULE D HYPERSOL VSMDH.72.595.05-30M,26060993,KRISH.SGR2,D,None


,Posting Date,Creation Time,Order No.,Material Desc.,Serial Number,Created By,Grade.,Reason
296711,2025-09-29,01:03:10,000000071405,PV MODULE D HYPERSOL VSMDH.72.595.05-30M,26354672,BISWAJIT.UD4,D,None
296712,2025-09-29,01:03:11,000000071405,PV MODULE D HYPERSOL VSMDH.72.595.05-30M,26354691,BISWAJIT.UD4,D,None
296713,2025-09-29,01:03:13,000000071405,PV MODULE D HYPERSOL VSMDH.72.595.05-30M,26354705,BISWAJIT.UD4,D,None
296714,2025-09-29,00:58:37,000000071405,PV MODULE D HYPERSOL VSMDH.72.595.05-30M,26355723,DIPANKAR.UD3,D,None
296715,2025-09-29,00:27:24,000000090391,PV MODULE D HYPERSOL VSMDH.72.595.05-30M,26352300,KRISH.SGR2,D,None


In [2]:
from pymongo import MongoClient
from datetime import datetime, time
import pandas as pd
import numpy as np

class BMongoDBManager:
    def __init__(self, db_name="b_grade_trend"):
        self.client = MongoClient('mongodb://localhost:27017/')
        self.db = self.client[db_name]
    
    def convert_to_mongo_compatible(self, obj):
        if obj is None:
            return None
        elif isinstance(obj, (int, float, str, bool)):
            return obj
        elif isinstance(obj, datetime):
            return obj
        elif isinstance(obj, time):
            return obj.isoformat()
        elif isinstance(obj, pd.Timestamp):
            return obj.to_pydatetime()
        elif pd.isna(obj):
            return None
        elif isinstance(obj, (np.integer, np.floating)):
            return float(obj) if isinstance(obj, np.floating) else int(obj)
        else:
            return str(obj)
    
    def get_collection_name(self, date_str):
        try:
            if isinstance(date_str, datetime):
                date_obj = date_str
            elif isinstance(date_str, pd.Timestamp):
                date_obj = date_str.to_pydatetime()
            else:
                date_obj = datetime.strptime(str(date_str), '%Y-%m-%d %H:%M:%S')
        except:
            try:
                date_obj = datetime.strptime(str(date_str), '%Y-%m-%d')
            except:
                print(f"Warning: Could not parse date: {date_str}")
                return None
        month_abbr = date_obj.strftime('%b').lower()
        year = date_obj.year
        return f"{month_abbr}_{year}"
    
    def prepare_document(self, row):
        document = {
            'posting_date': self.convert_to_mongo_compatible(row.get('Posting Date')),
            'creation_time': self.convert_to_mongo_compatible(row.get('Creation Time')),
            'order_no': self.convert_to_mongo_compatible(row.get('Order No.')),
            'material_desc': self.convert_to_mongo_compatible(row.get('Material Desc.')),
            'serial_number': self.convert_to_mongo_compatible(row.get('Serial Number')),
            'created_by': self.convert_to_mongo_compatible(row.get('Created By')),
            'grade': self.convert_to_mongo_compatible(row.get('Grade.')),
            'reason': self.convert_to_mongo_compatible(row.get('Reason')),
            'processing_date': datetime.now()
        }
        return document
    
    def insert_dataframe_to_mongo(self, dataframe):
        records_processed = 0
        collections_used = set()
        errors = []        
        for index, row in dataframe.iterrows():
            try:
                document = self.prepare_document(row)
                collection_name = self.get_collection_name(row.get('Posting Date'))
                if not collection_name:
                    errors.append(f"Row {index}: Invalid date format")
                    continue
                collection = self.db[collection_name]
                result = collection.insert_one(document)
                if result.inserted_id:
                    records_processed += 1
                    collections_used.add(collection_name)                
            except Exception as e:
                error_msg = f"Error processing row {index}: {e}"
                errors.append(error_msg)
                print(error_msg)
                continue
        print(f"\nMongoDB Storage Summary:")
        print(f"Records successfully processed: {records_processed}")
        print(f"Collections updated/created: {list(collections_used)}")
        if errors:
            print(f"\nErrors encountered: {len(errors)}")
            for error in errors[:5]:
                print(f"  - {error}")
            if len(errors) > 5:
                print(f"  ... and {len(errors) - 5} more errors")
        return records_processed, collections_used, errors
    
    def get_monthly_data(self, month_abbr, year):
        collection_name = f"{month_abbr.lower()}_{year}"
        if collection_name in self.db.list_collection_names():
            collection = self.db[collection_name]
            return list(collection.find({}, {'_id': 0}))
        else:
            return []
    
    def list_all_collections(self):
        return self.db.list_collection_names()
    
    def get_collection_stats(self, collection_name):
        if collection_name in self.db.list_collection_names():
            collection = self.db[collection_name]
            count = collection.count_documents({})
            sample_docs = list(collection.find().limit(2))
            return count, sample_docs
        return 0, []

def display_database_stats(mongo_manager, collections_list):
    print("\nCollection Statistics:")
    print("-" * 50)    
    for collection_name in collections_list:
        count, sample_docs = mongo_manager.get_collection_stats(collection_name)
        print(f"Collection '{collection_name}': {count} documents")        
        if sample_docs:
            print(f"Sample document from {collection_name}:")
            for doc in sample_docs:
                doc_display = doc.copy()
                doc_display['_id'] = str(doc_display['_id'])
                print(f"  - {doc_display}")
        print()

mongo_manager = BMongoDBManager()
records_processed, collections_used, errors = mongo_manager.insert_dataframe_to_mongo(processed_df)
display_database_stats(mongo_manager, collections_used)


MongoDB Storage Summary:
Records successfully processed: 296716
Collections updated/created: ['sep_2025']

Collection Statistics:
--------------------------------------------------
Collection 'sep_2025': 296716 documents
Sample document from sep_2025:
  - {'_id': '691465d42ab5720523856b82', 'posting_date': datetime.datetime(2025, 9, 1, 0, 0), 'creation_time': '08:56:26', 'order_no': '000000071361', 'material_desc': 'PV MODULE D HYPERSOL VSMDH.72.600.05-30M', 'serial_number': 26062065, 'created_by': 'BISWAJIT.UD4', 'grade': 'D', 'reason': None, 'processing_date': datetime.datetime(2025, 11, 12, 16, 17, 48, 262000)}
  - {'_id': '691465d42ab5720523856b83', 'posting_date': datetime.datetime(2025, 9, 1, 0, 0), 'creation_time': '15:31:30', 'order_no': '000000090342', 'material_desc': 'PV MODULE D HYPERSOL VSMDH.72.595.05-30M', 'serial_number': 26060983, 'created_by': 'KRISH.SGR2', 'grade': 'D', 'reason': None, 'processing_date': datetime.datetime(2025, 11, 12, 16, 17, 48, 300000)}

